In [ ]:
from trainyourfly.data.data_processing import DataProcessor
from trainyourfly.connectome_models.graph_models import FullGraphModel
from configs import config
from trainyourfly.utils.utils import get_image_paths

TRAIN_IMAGES_DIR = "images/ans/train"
train_images = get_image_paths(TRAIN_IMAGES_DIR)

# Initialize the data processor (loads connectome, sets up Voronoi tessellation)
data_processor = DataProcessor(config)

# Create the connectome-based model
model = FullGraphModel(data_processor, config).to(config.DEVICE)

# Process a batch of images
images, labels = data_processor.get_data_from_paths(image_paths)
inputs, labels = data_processor.process_batch(images, labels)

# Forward pass through the connectome
outputs = model(inputs)

ModuleNotFoundError: No module named 'torch'